In [1]:
# WARNING!!
# This notebook is not updated after the last model calibrations refactorization

# MATLAB model
import os
os.environ["MR"] = f"{os.environ['HOME']}/PSA/MATLAB_runtime"
MR = os.environ["MR"]
os.environ["LD_LIBRARY_PATH"] = f"{MR}/v911/runtime/glnxa64:{MR}/v911/bin/glnxa64:{MR}/v911/sys/os/glnxa64:{MR}/v911/sys/opengl/lib/glnxa64"

# print(os.environ["MR"])
# print(os.environ["LD_LIBRARY_PATH"])

# Add parent directory to path
import sys
sys.path.append("..")

# General
import pandas as pd
import numpy as np
from utils.constants import vars_info
import logging

# Visualization
import plotly
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from IPython.display import display, HTML, Markdown
import plotly.colors as pc

plotly.offline.init_notebook_mode()

display(
    HTML('<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>')
)

df_pc = pc.DEFAULT_PLOTLY_COLORS

%load_ext autoreload

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
from iapws import IAPWS97 as w_props

# Load data
data = pd.read_csv('../data/20230807_aquasol.csv', parse_dates=['TimeStamp'], date_format='%d-%b-%Y %H:%M:%S')

var_ids = ["Tamb", "Thx_p_in", "Thx_p_out", "Thx_s_in", "Thx_s_out", "mhx_p", "mhx_s"]
signal_ids = [vars_info[var_id]["signal_id"] for var_id in var_ids]

# Out of all the data, get a subset
data = data[signal_ids + ['TimeStamp']]

# Rename columns
data.rename(columns={signal_id: var_id for var_id, signal_id in zip(var_ids, signal_ids)}, inplace=True)
data.rename(columns={'TimeStamp': 'time'}, inplace=True)

# Resample data
data.set_index('time', inplace=True)
data = data.resample('1min').mean()

# Filter nan
data = data.dropna()

# data.set_index('time', inplace=True)

# Convert units to common units

# L/min -> m3/h
# rho_sf = np.array([w_props(P=0.1, T=(Tsf_out+Tsf_in)/2+273.15).rho for Tsf_out, Tsf_in in zip(data['Tsf_out'], data['Tsf_in'])]) # kg/m3
data['mhx_p'] = data['mhx_p'] / 1000 * 60
data['mhx_s'] = data['mhx_s'] / 1000 * 60


# Filter invalid temperatures
print(sum(data['Thx_p_out'] < data['Thx_s_out']))

data.loc[ data['Thx_s_out'] > data['Thx_p_out'], 'Thx_s_out' ] = data['Thx_p_out']-0.5
print(sum(data['Thx_p_out'] < data['Thx_s_out']))

In [3]:
data.head()

In [4]:
# Visualize data

def visualize_heat_exchanger(df, include_prediction=False, UA=None, H=None, IAE=None):

    # display(df.head())

    color_blue = 'rgba(53, 132, 228, 0.5)'
    color_red  = 'rgba(224, 27, 36, 0.5)'

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                        subplot_titles=['Temperatures evolution', 'Flows'],)
    
    fig.print_grid()
    
    # Temperatures
    row_idx = 1
    var_ids = [['Thx_p_in', 'Thx_p_out'], ['Thx_s_out', 'Thx_s_in']]
    for var_id, color, pattern in zip(var_ids, [color_red, color_blue], ['\\', '/']):
        # Add filled area between Thx_p_in and Thx_p_out with reddish color
        fig.add_trace(
            go.Scatter(
                x=df.index, 
                y=df[var_id[0]], 
                # fill='tonexty', 
                line=dict(color=color, width=0.5), 
                mode='lines', 
                fillcolor=color, 
                name=vars_info[var_id[0]]["label_plotly"]
                ),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=df.index, 
                    y=df[var_id[1]], 
                    fill='tonexty', 
                    mode='lines',
                    line=dict(color=color, width=0.5), 
                    fillcolor=color,
                    fillpattern=pattern,
                    name=vars_info[var_id[1]]["label_plotly"]),
            row=1, col=1
        )
    fig.update_yaxes(title_text=vars_info[var_id[0]]["units_model"],row=row_idx, col=1)
    
    
    if include_prediction:
        var_ids = ['Thx_p_out_pred', 'Thx_s_out_pred']
        labels = ['T<sub>hx,p,out</sub> prediction', 'T<sub>hx,s,out</sub> prediction']
        row_idx = 1
        for var_id, color, label in zip(var_ids, [color_red, color_blue], labels):
            fig.add_trace(
                go.Scatter(
                    x=df.index, 
                    y=df[var_id], 
                    name=label,  
                    line=dict(color=color, width=4, dash='dash'),          
                ), 
                row=row_idx, col=1
            )
        
    # Flows
    var_ids = ['mhx_p', 'mhx_s']
    row_idx = 2
    for var_id, color in zip(var_ids, [color_red, color_blue]):
        fig.add_trace(
            go.Scatter(
                x=df.index, 
                y=df[var_id], 
                name=vars_info[var_id]["label_plotly"],  
                line=dict(color=color, width=2),          
            ), 
            row=row_idx, col=1
        )
    fig.update_yaxes(title_text=vars_info[var_id]["units_model"], row=row_idx, col=1)

        
    # Position legend at the bottom right
    fig.update_layout(
        # legend=dict(
        #     orientation="v",
        #     yanchor="bottom",
        #     xaxis_title='Time',
        #     y=0.02,
        #     xanchor="right",
        #     x=1.15
        # ),
        legend=dict(x=0, y=1, bgcolor='rgba(255, 255, 255, 0.5)'),
        height=800,
        title=f'Heat exchanger model, UA={UA:.2e} W/K, H={H:.2f}, IAE={IAE:.2f}' if UA is not None else 'Heat exchanger evolution',
    )
        
    return fig    

In [5]:
vars_info['Thx_p_in']

In [6]:
vars_info['Thx_s_out']

In [7]:
fig = visualize_heat_exchanger(data.iloc[80:-200], include_prediction=False)

fig.show()


In [8]:
# Run model
import numpy as np

# Model
from models import heat_exchanger_model
%autoreload 2

df = data.copy()

df = df.iloc[80:]
df = df.where(df['Thx_p_in']-df['Thx_p_out']>1).dropna()
Tp_in = df['Thx_p_in'].values
Ts_in = df['Thx_s_in'].values
Qp = df['mhx_p'].values
Qs = df['mhx_s'].values
Tamb = df['Tamb'].values

Tp_out = np.zeros(len(Tp_in))
Ts_out = np.zeros(len(Tp_in))
for i in range(1, len(Tp_in)):
    Tp_out[i], Ts_out[i] = heat_exchanger_model(Tp_in[i], Ts_in[i], Qp[i], Qs[i], Tamb[i], UA=4877.039, H=0.05)

df['Thx_p_out_pred'] = Tp_out
df['Thx_s_out_pred'] = Ts_out

# display(df.head())

fig = visualize_heat_exchanger(df, include_prediction=True)
fig.show()

In [9]:
df.head()


In [10]:
import math

i = 200

# Parameters
UA = 28000

# Inputs
Tp_in = df['Thx_p_in'][i]
Ts_in = df['Thx_s_in'][i]
Qp = df['mhx_p'][i]
Qs = df['mhx_s'][i]

print(f'Tp_in: {Tp_in}, Ts_in: {Ts_in}, Qp: {Qp}, Qs: {Qs}')

w_props_Tp_in = w_props(P=0.16, T=Tp_in+273.15)
w_props_Ts_in = w_props(P=0.1, T=Ts_in+273.15)
cp_Tp_in = w_props_Tp_in.cp*1e3 # P=1 bar->0.1 MPa C, cp [KJ/kg·K] -> [J/kg·K]
cp_Ts_in = w_props_Ts_in.cp*1e3 # P=1 bar->0.1 MPa C, cp [KJ/kg·K] -> [J/kg·K]

print(f'rho_p = {w_props_Tp_in.rho:.2f}, rho_s = {w_props_Ts_in.rho:.2f}')

mp = Qp/3600*w_props_Tp_in.rho # rho [kg/m³] # Convertir m^3/s a kg/s
ms = Qs/3600*w_props_Ts_in.rho # rho [kg/m³] # Convertir m^3/s a kg/s

print(f'cp_Tp_in: {cp_Tp_in}, cp_Ts_in: {cp_Ts_in}, mp: {mp}, ms: {ms}')

mcp_min = min([mp*cp_Tp_in, ms*cp_Ts_in])
mcp_max = max([mp*cp_Tp_in, ms*cp_Ts_in])

print(f'mcp_min: {mcp_min}, mcp_max: {mcp_max}')

theta = UA*(1/mcp_max-1/mcp_min)

eta_p = (1-math.e**theta)/( 1-math.e**theta*(mcp_min/mcp_max) )
eta_s = mp*cp_Tp_in/(ms*cp_Ts_in) 

print(f'theta: {theta} eta_p: {eta_p}, eta_s: {eta_s}')

Tp_out = Tp_in - eta_p*(mcp_min)/(mp*cp_Tp_in)*(Tp_in-Ts_in) # ºC
Ts_out = Ts_in + eta_s*(Tp_in-Tp_out) # ºC

print(f'Tp_out: ref {df["Thx_p_out"][i]} mod {Tp_out}, Ts_out: ref {df["Thx_s_out"][i]} mod {Ts_out}')


In [11]:
np.column_stack((Tp_out_ref, Ts_out_ref()))

In [12]:
##%% Parameter fit
import numpy as np
from utils.parameters_fit import objective_function
from models import heat_exchanger_model
# from optimparallel import minimize_parallel
from scipy.optimize import minimize
from utils.parameters_fit import calculate_iae, calculate_ise, calculate_itae

# The calibration should only be performed in the range where the field is operating in normal conditions
# df = df.where(df['Tsf_out']-df['Tsf_in']>1).dropna()

# The calibration should only be performed in the range where the field is operating in normal conditions
df = df.where(df['Thx_p_in']-df['Thx_p_out']>1).dropna()
df = df.where(df['Thx_s_in']>65).dropna()

fig = visualize_heat_exchanger(df, include_prediction=False)
fig.show()

# Parameters

# Inputs 
Tp_in = df['Thx_p_in'].values
Ts_in = df['Thx_s_in'].values
Qp = df['mhx_p'].values
Qs = df['mhx_s'].values
Tamb = df['Tamb'].values

# Tamb = df['Tamb'].values

# msrc = np.zeros(len(data)-1, dtype=float); mdis = np.zeros(len(data)-1, dtype=float)
# for idx in range(1, len(data)-1):
#     msrc[idx] = Qsrc[idx]/60*w_props(P=0.1, T=Tt_in[idx]+273.15).rho*1e-3 # rho [kg/m³] # Convertir L/min a kg/s
#     mdis[idx] = Qdis[idx]*w_props(P=0.1, T=data.Tts_h_t[idx]+273.15).rho*1e-3 # rho [kg/m³] # Convertir L/s a kg/s

# Experimental outputs
Tp_out_ref = df['Thx_p_out'].values
Ts_out_ref = df['Thx_s_out'].values

# Define optimizer inputs
inputs = [Tp_in, Ts_in, Qp, Qs, Tamb]  # Input values
outputs = np.column_stack((Tp_out_ref, Ts_out_ref))  # Actual output values
params = ()    # Constant model parameters
params_objective_function = {'metric': 'IAE', 'recursive':False, 'n_outputs':2, 
                             'n_parameters': 2} # 'len_outputs':[N, N]

# Set initial parameter values
initial_parameters = [2800, 0]
# initial_parameters = [0.85, 0.85]

#         beta min, beta max    Hmin,    Hmax
bounds = ((1500, 5000), (0,1))
# bounds = ((0, 10), (0, 10))

# Perform parameter calibration
optimized_parameters = minimize(
    objective_function,
    initial_parameters,
    args=(heat_exchanger_model, inputs, outputs, params, params_objective_function),
    bounds = bounds,
    method='Powell'
).x

UA = optimized_parameters[0]
H = optimized_parameters[1]

# eta_p = optimized_parameters[0]
# eta_s = optimized_parameters[1]

# optimized_parameters = 
# Run model with optimized parameters

In [13]:
# Initialize result vectors
Tp_out_mod = np.zeros(len(df), dtype=float)
Ts_out_mod = np.zeros(len(df), dtype=float)

# Evaluate model
for i in range(len(df)):
    Tp_out_mod[i], Ts_out_mod[i] = heat_exchanger_model(Tp_in[i], Ts_in[i], Qp[i], Qs[i], Tamb[i], UA=UA, H=H)
    # Tp_out_mod[i], Ts_out_mod[i] = heat_exchanger_model(Tp_in[i], Ts_in[i], Qp[i], Qs[i], Tamb[i], eta_p, eta_s)

out_mod = np.column_stack((Tp_out_mod, Ts_out_mod))    
# Calculate performance metrics
iae  = calculate_iae(out_mod, outputs)
ise  = calculate_ise(out_mod, outputs)
itae = calculate_itae(out_mod, outputs)

# Visualize result
df['Thx_p_out_pred'] = Tp_out_mod
df['Thx_s_out_pred'] = Ts_out_mod

fig = visualize_heat_exchanger(df, include_prediction=True, UA=UA, H=H, IAE=iae)
# fig = visualize_heat_exchanger(df, include_prediction=True, UA=eta_p, H=eta_s, IAE=iae)

fig.show()

In [13]:
UA

In [14]:
fig.write_image("heat_exchanger_model.png", scale=2)